# Descarga de datos

In [2]:
from google.colab import output
!pip install gdown
!gdown https://drive.google.com/uc?id=1L5kNJEwRRlNtBvXVt4KZB0EiTQk23iaV # pokedex_processed.csv
!gdown https://drive.google.com/uc?id=19dw2vVgF6pht_-GF5_IvxrVLOSkT8kCQ # pokedex_processed.json
output.clear()

# Datos

Los datos corresponden al mundo de _Pokémon_ separado en dos bases de datos: Una relacional SQL (PostgreSQL) y otra de documentos NoSQL (MongoDB). Ambas son levantadas dentro del entorno de ejecución del notebook en Google Colab.

A continuación se crea y describe a cada una:

# PostgreSQL

Cada columna de la tabla de pokedex se describe a continuación:

```text
###################
## Tabla pokedex ##
###################

Id           : (int) Identificador único en el Pokédex
Name         : (str) Nombre del Pokémon
Category     : (str) La "especie" del Pokémon (en Pokémon Sol y Luna)
Height (m)   : (int) Altura en metros
Weight (kg)  : (int) Masa en kilogramos
Capture Rate : (int) Cuantifica la probabilidad de capturar al Pokémon (a mayor valor, más probable).
Egg Steps    : (int) Cuantifica el número de pasos que requiere en el juego para salir de un huevo.
Exp Group    : (str) Categoría que determina cuanta experiencia necesita para subir de nivel.
HP           : (int) Vida
Attack       : (int) Ataque
Defense      : (int) Defensa
Sp. Attack   : (int) Ataque Especial
Sp. Defense  : (int) Defensa Especial
Speed        : (int) Velocidad

```

In [3]:
# install
!apt update
!apt install postgresql postgresql-contrib &>log
!service postgresql start
!sudo -u postgres psql -c "CREATE USER root WITH SUPERUSER"
# pin package versions for google colab compatibility
!pip install SQLAlchemy==1.4.46
!pip install ipython-sql==0.4.1
# set connection
%load_ext sql
%config SqlMagic.feedback=False
%config SqlMagic.autopandas=True
%config SqlMagic.displaycon = False
%sql postgresql+psycopg2://@/postgres
output.clear()

In [ ]:
import pandas as pd
from sqlalchemy import create_engine
engine = create_engine("postgresql+psycopg2://@/postgres", echo=False)
df_pokedex = pd.read_csv("pokedex_processed.csv", index_col=False)
df_pokedex.to_sql("pokedex", con=engine, if_exists="replace", index=False)

In [5]:
%%sql
SELECT *
FROM pokedex
LIMIT 10

,Id,Name,Category,Height (m),Weight (kg),Capture Rate,Egg Steps,Exp Group,HP,Attack,Defense,Sp. Attack,Sp. Defense,Speed
0,1,Bulbasaur,Seed Pokémon,0.7,6.9,45,5120,Medium Slow,45,49,49,65,65,45
1,2,Ivysaur,Seed Pokémon,1.0,13.0,45,5120,Medium Slow,60,62,63,80,80,60
2,3,Venusaur,Seed Pokémon,2.0,100.0,45,5120,Medium Slow,80,82,83,100,100,80
3,4,Charmander,Lizard Pokémon,0.6,8.5,45,5120,Medium Slow,39,52,43,60,50,65
4,5,Charmeleon,Flame Pokémon,1.1,19.0,45,5120,Medium Slow,58,64,58,80,65,80
5,6,Charizard,Flame Pokémon,1.7,90.5,45,5120,Medium Slow,78,84,78,109,85,100
6,7,Squirtle,Tiny Turtle Pokémon,0.5,9.0,45,5120,Medium Slow,44,48,65,50,64,43
7,8,Wartortle,Turtle Pokémon,1.0,22.5,45,5120,Medium Slow,59,63,80,65,80,58
8,9,Blastoise,Shellfish Pokémon,1.6,85.5,45,5120,Medium Slow,79,83,100,85,105,78
9,10,Caterpie,Worm Pokémon,0.3,2.9,255,3840,Medium Fast,45,30,35,20,20,45


## MongoDB

La colección pokedex tiene documentos con 4 entradas: Un identificador único del pokémon y arreglos con información acerca del mismo. A continuación se detalla cada uno:

```text
#######################
## Colección pokedex ##
#######################

"pokemon_id": : (int)       Número identificador único en el Pokédex
"abilities"   : (List[str]) Lista de las habilidades del Pokémon
"moves"       : (List[str]) Lista de los movimientos del Pokémon
"types"       : (List[str]) Lista de los tipos del Pokémon
```

In [6]:
!apt install mongodb
!service mongodb start
!pip install pymongo
!mongoimport --db local --collection pokedex --jsonArray < 'pokedex_processed.json'
output.clear()

In [7]:
from pymongo import MongoClient
client = MongoClient()
client.list_database_names() # ['admin', 'local']
db = client['local']
collection = db['pokedex']

Para tener una idea de cómo se ven los datos en la colección, aquí les dejamos impresión del primer Pokemon en la colección:

In [8]:
import pprint
for pokemon in collection.find({}).limit(1):
    pprint.pprint(pokemon)

{'_id': ObjectId('648a420b82e29c2db100d267'),
 'abilities': ['Overgrow', 'Chlorophyll'],
 'moves': ['Tackle is described as A physical attack in which the user charges '
           'and slams into the target with its whole body.',
           'Growl is described as The user growls in an endearing way, making '
           'opposing Pokémon less wary. This lowers their Attack stat.',
           'Vine Whip is described as The target is struck with slender, '
           'whiplike vines to inflict damage.',
           'Poison Powder is described as The user scatters a cloud of '
           'poisonous dust that poisons the target.',
           'Sleep Powder is described as The user scatters a big cloud of '
           'sleep-inducing dust around the target.',
           'Take Down is described as A reckless, full-body charge attack for '
           'slamming into the target. This also damages the user a little.',
           'Razor Leaf is described as Sharp-edged leaves are launched to '
    

# Preguntas

Para resolver estas preguntas será necesario que consultes a `PostgreSQL` y/o `MongoDB`.

En este `notebook` puedes almacenar el resultado de una consulta `PostgreSQL` en una variable como un `pandas.DataFrame` de la siguiente forma:

```python
df = %sql SELECT "Name" FROM pokedex WHERE "Id" = 99;
```

## Pregunta 1
Entregue el nombre (name) y la cantidad de habilidades (abilites) de los 10 pokemones más pesados en kilogramos

## Pregunta 2

Nombre los 3 pokemones con más movimientos. Se recomienda revisar la documentación de MongoDB, particularmente de [`$sort`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/sort/) y [`$limit`](https://www.mongodb.com/docs/manual/reference/operator/aggregation/limit/).

## Pregunta 3

Cuál es el elemento de los pokemones más densos? Indique los 5 elementos de pokemones más densos. Asuma que la densidad es el peso dividido en la altura, y que los pokemones tienen un solo tipo (ignore todos los elementos de cada pokemon excepto uno).

## Pregunta 4

Vea si existe alguna correlación entre algún atributo (ataque, defensa, ataque especial, defensa especial, velocidad y HP) y la cantidad de movimientos que tienen los pokemones.

Hay algun otro par de variables (de las mencionadas anteriormente) que tengan mayor correlación entre sí? Indique el par de atributos que tengan la mayor correlación entre sí y haga un gráfico de dispersión para visualizar la correlación.

Los atributos con mayor correlación son YY y ZZ.

## Importante:
Para las últimas preguntas será necesario que crees y consultes un índice de texto para el arreglo de strings `moves`.

In [ ]:
# Aquí escribe el código para crear el índice de texto


## Pregunta 5
Entregue el nombre de todos los pokemones los cuales en sus movimientos contengan las palabras `fire` y `thunder` y tengan un Ataque Especial (Sp. Attack) entre `110` y `130` (ambos valores incluidos)

## Pregunta 6
Entregue el nombre junto con la suma de las estadísticas de combate (HP, Attack, Defense, Sp. Attack, Sp. Defense y Speed) de los pokemones que en sus movimientos contengan las palabras `ice`, `freeze`, `snow` o `cold`, **pero no** las palabras `water` y `wind`. Deben estar ordenado alfabéticamente por su nombre.